In [1]:
import sklearn
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [5]:
def hypotes(X,w):
    return np.dot(X,w)

In [7]:
def loss_func(X,y,w):
    return np.square(hypotes(X,w)-y).sum()/2*X.size()

In [9]:
def gradient_step(X,y,w,learn_rate):
    m=len(y)
    grad = (X.T @ (hypotes(X,w)-y))/m
    w-=learn_rate*grad
    return w

In [55]:
df = pd.read_csv("./Housing.csv")

In [57]:
df.info()
df.head(6)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished


In [59]:
df= df.iloc[:,:4]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   price      545 non-null    int64
 1   area       545 non-null    int64
 2   bedrooms   545 non-null    int64
 3   bathrooms  545 non-null    int64
dtypes: int64(4)
memory usage: 17.2 KB


In [93]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=df[["area", "bathrooms", "bedrooms"]].to_numpy()
y=df["price"].to_numpy()
Xs = scaler.fit_transform(X)
print(Xs)

[[ 1.04672629  1.42181174  1.40341936]
 [ 1.75700953  5.40580863  1.40341936]
 [ 2.21823241  1.42181174  0.04727831]
 ...
 [-0.70592066 -0.57018671 -1.30886273]
 [-1.03338891 -0.57018671  0.04727831]
 [-0.5998394  -0.57018671  0.04727831]]


In [95]:
scaler.mean_, scaler.scale_

(array([5.15054128e+03, 1.28623853e+00, 2.96513761e+00]),
 array([2.16814915e+03, 5.02008423e-01, 7.37386427e-01]))

In [97]:
w = np.zeros(Xs.shape[1])

In [99]:
learn_rate = 0.01
w = gradient_step(Xs, y, w, learn_rate)

In [101]:
predict_price = hypotes(Xs,w)

In [103]:
from sklearn.metrics import mean_squared_error, r2_score
# MSE
# mean_squared_error = np.sum((y - y_hat.T)**2) / len(y)

# The mean squared error
print(f"MSE one gradient step: {mean_squared_error(y, predict_price)}")

MSE one gradient step: 26166027517939.773


In [73]:
from sklearn.linear_model import LinearRegression
model= LinearRegression().fit(Xs, y)

In [75]:
model.__dict__

{'fit_intercept': True,
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'n_features_in_': 3,
 'coef_': array([821214.14349519, 695808.52272537, 299983.57107963]),
 'rank_': 3,
 'singular_': array([28.54332915, 21.92450924, 18.42808334]),
 'intercept_': 4766729.247706422}

In [89]:
y_pred=model.predict(Xs)
print(f"MSE sklearn: {mean_squared_error(y, y_pred)}")


MSE sklearn: 1791170049977.3193


Analytical method

In [107]:
# Add a column with 1 for bias
Xb_scaled = np.hstack((np.ones((Xs.shape[0], 1)), Xs))

# Beregn den inverse av produktet av transponert Xb_scaled og Xb_scaled
z = np.linalg.pinv(np.dot(Xb_scaled.T, Xb_scaled))

# Beregn parameterne ved hjelp av normal-ligningen
w_analytic = np.dot(z, np.dot(Xb_scaled.T, y))

print("Parameters (w):", w_analytic)

Parameters (w): [4766729.24770642  821214.14349519  695808.52272537  299983.57107963]


In [109]:
np.hstack((np.ones((Xs.shape[0], 1)), Xs))

array([[ 1.        ,  1.04672629,  1.42181174,  1.40341936],
       [ 1.        ,  1.75700953,  5.40580863,  1.40341936],
       [ 1.        ,  2.21823241,  1.42181174,  0.04727831],
       ...,
       [ 1.        , -0.70592066, -0.57018671, -1.30886273],
       [ 1.        , -1.03338891, -0.57018671,  0.04727831],
       [ 1.        , -0.5998394 , -0.57018671,  0.04727831]])

In [111]:
np.zeros(Xb_scaled.shape[1])

array([0., 0., 0., 0.])

In [113]:
np.linalg.inv(np.dot(Xb_scaled.T, Xb_scaled))

array([[ 1.83486239e-03, -2.54238233e-19,  2.06461382e-18,
        -1.57537740e-20],
       [-2.54238233e-19,  1.92110484e-03, -3.06052072e-04,
        -1.77293949e-04],
       [ 2.06461382e-18, -3.06052072e-04,  2.18188122e-03,
        -7.69394754e-04],
       [-1.57537740e-20, -1.77293949e-04, -7.69394754e-04,
         2.14948594e-03]])

In [115]:
np.dot(z, np.dot(Xb_scaled.T, y))

array([4766729.24770642,  821214.14349519,  695808.52272537,
        299983.57107963])

In [121]:

def hypotes1(X,w):
    return np.dot(X,w[1:])

predict_price = hypotes1(Xs, w_analytic)

In [125]:
print(mean_squared_error(y, predict_price))

24512877770917.152
